In [1]:
%pip install sympy numpy
# Importamos `sympy` para que nos ayude a realizar todas las operaciones 
# algebraicas
import sympy, numpy


Note: you may need to restart the kernel to use updated packages.


1.1) Si se usan 8 bits para representar los enteros positivos y negativos en \
complemento a dos, cuál es el entero positivo más grande y el negativo más \
pequeño (en magnitud) en decimal?

In [2]:
def twos_complement(val, bits):
    if (val & (1 << (bits - 1))) != 0:
        val = val - (1 << bits)
    return val


min_negative = 0b10000000
max_positive = 0b01111111
bit_length = 8

print(f'El negativo más pequeño en magnitud posible en complemento a dos es: '
      f'{min_negative:>08b},\n que termina representado en decimal como: '
      f'{twos_complement(min_negative, bit_length)}.')
print(f'El positivo más grande en magnitud posible en complemento a dos es: '
      f'{max_positive:>08b},\n que termina representado en decimal como: '
      f'{twos_complement(max_positive, bit_length)}.')


El negativo más pequeño en magnitud posible en complemento a dos es: 10000000,
 que termina representado en decimal como: -128.
El positivo más grande en magnitud posible en complemento a dos es: 01111111,
 que termina representado en decimal como: 127.


1.3) Hallar el epsilon de la máquina para una IBM PC en WATFOR-77.

Originalmente se planeó utilizar Python dado el entorno de entrega, pero luego \
se buscaron formas para utilizar Fortran.

Se escribe a través de comandos bash a un archivo `.f95`

In [3]:
%%writefile get_epsilon.f95

PROGRAM get_epsilon
    real :: E
    E = 1.0
    1 IF( E+1.0>1.0 ) THEN
        E=E/2.0
        GOTO 1
    END IF
    PRINT*, "EPSILON FOUND: ", E
END program get_epsilon

Overwriting get_epsilon.f95


Compilamos con `gfortran` y ejecutamos el archivo compilado

In [4]:
%%bash

gfortran -ffree-form get_epsilon.f95
./a.out

 EPSILON FOUND:    5.96046448E-08


También se calculó utilizando `python` base y `numpy`:

In [5]:
def get_epsilon() -> float:
    e = 1
    while e+1>1:
        e = e/2
    return e

print(f'EPSILON with base Python: {get_epsilon()}')
print(f'EPSILON with Numpy: {numpy.finfo(float).eps}')

EPSILON with base Python: 1.1102230246251565e-16
EPSILON with Numpy: 2.220446049250313e-16


Podemos ver que los valores varían, pero esto puede ser dado a la forma en que \
el compilador de fortran y el interprete de Python generan las operaciones para \
realizar los calculas, además de Fortran ser un lenguaje de muy poco uso y \
mantenimiento/desarrollo en la actualidad.

La diferencia entre Numpy y Python base puede ser debido a cómo Numpy obtiene \
esa información, de lo que realmente no tenemos información.

1.5) Evalúe `exp(x) - 1` para `x = 0.0001`, aplicando el desarrollo de Taylor 
para `exp(x)`. Use los primeros tres términos.

In [6]:
sympy.init_printing()

x, a = sympy.symbols('x a')
fa = sympy.exp(a)
d1a = sympy.diff(fa)
d2a = sympy.diff(d1a)
d3a = sympy.diff(d2a)
taylor = fa + (d1a/sympy.factorial(1)*(x-a)) + (d2a/sympy.factorial(2)*(x-a)**2) + (d3a/sympy.factorial(3)*(x-a)**3)

Desarrollando la serie de taylor para los primeros 3 términos tenemos:

In [7]:
sympy.Eq(sympy.exp(x), taylor)

             3  a           2  a                   
 x   (-a + x) ⋅ℯ    (-a + x) ⋅ℯ              a    a
ℯ  = ──────────── + ──────────── + (-a + x)⋅ℯ  + ℯ 
          6              2                         

Asumiendo que la serie de Taylor la desarrollamos alrededor de 0 para exp(x). Tenemos:

In [8]:
taylor_around_zero = taylor.subs(a,0)
sympy.Eq(sympy.exp(x), taylor_around_zero)


      3    2        
 x   x    x         
ℯ  = ── + ── + x + 1
     6    2         

Evaluando `x=0.0001` nos queda:

In [9]:
dummy_value = sympy.symbols('0.0001')
not_solved_taylor = sympy.Eq(sympy.exp(x), taylor_around_zero.subs(x,dummy_value))
not_solved_taylor

           3         2             
 x   0.0001    0.0001              
ℯ  = ─────── + ─────── + 0.0001 + 1
        6         2                

In [10]:
taylor_solution = taylor_around_zero.subs(x,.0001)
sympy.Eq(sympy.exp(x), taylor_solution)

 x                   
ℯ  = 1.00010000500017

Y evaluando directamente la función `exp(x)` con `x = 0.0001` utilizando un \
número de alta precisión de numpy tenemos:

In [11]:
high_precision_solution = numpy.float128(numpy.exp(.0001))
high_precision_solution

1.0001000050001667141

Donde podemos ver una diferencia de:

In [12]:
1.00010000500017 - high_precision_solution

3.3306690738754696213e-15

Realizando la operación inicialmente propuesta `exp(x) - 1` tenemos:

In [13]:
sympy.Eq(sympy.exp(x), taylor_solution - 1)

 x                       
ℯ  = 0.000100005000166714

In [14]:
high_precision_solution - 1

0.00010000500016671409753

1.9 a) Si Ia siguiente función se escribe en un programa, ¿en cuál rango de x aparecerá un desborde o una división entre cero originados por el error de redondeo?

$f(x) = \frac{1}{1-\tanh(x)}$

Suponga que el número positivo más pequeño es $3\times 10^{-39}$ y el epsilon de la máquinaes $1.2 \times 10 ^{-7} $

b) Reescriba la ecuación de tal forma que no se necesite restar.

a) El desborde aparecería cuando el valor de $\tanh(x)$ se encuentre en el rango de $(1-3\times10^{-39}) \times \epsilon$. Lo que vendría siendo $\tanh(x) = 1.7\times 10 ^{-7}$. Realizando este experimento en el computador podemos acercarnos lo más posible al valor necesario de $x$ para que $\tanh(x)$ empezando con $x = 19$ ya que los incrementos de épsilon son muy pequeños y tardaría mucho para un entero menor que esto.

Con esto podemos obtener un valor de $x$ anterior al redondeo de:

In [15]:
E = 1.2e-7
minNumb = 3e-39
i = 19
while 1 - numpy.tanh(i) > minNumb:
  i+=E

sympy.Eq(sympy.symbols('x'), i-E)

x = 18.99999988

Multiplicamos por $ϵ$ para obtener el rango de:

In [16]:
sympy.Eq(sympy.symbols('range'), (i-E)*E)

range = 2.2799999856e-6

b) Sabiendo que podemos hacer la igualdad de:

In [17]:
x = sympy.symbols('x')

tanhRel = (sympy.exp(x) - sympy.exp(-x))/(sympy.exp(x) + sympy.exp(-x))
sympy.Eq(sympy.tanh(x), tanhRel)

           x    -x
          ℯ  - ℯ  
tanh(x) = ────────
           x    -x
          ℯ  + ℯ  

Reemplazando la ecuación original de $f(x) = \frac{1}{1-\tanh(x)}$ con la relación anterior obtenemos:

In [18]:
originalExp = 1 / (1 - sympy.tanh(x))

replacedExp = originalExp.subs(sympy.tanh(x), tanhRel)
sympy.Eq(sympy.symbols('f(x)'), replacedExp)

             1       
f(x) = ──────────────
          x    -x    
         ℯ  - ℯ      
       - ──────── + 1
          x    -x    
         ℯ  + ℯ      

Finalmente simplificando dicha relación nos queda:

In [19]:
sympy.Eq(sympy.symbols('f(x)'), replacedExp.simplify())

        2⋅x    
       ℯ      1
f(x) = ──── + ─
        2     2

Lo cual elimina todas las restas de la ecuación.